# Azure AI Agents with Semantic Kernel Plugins - Complete Tutorial

🔧 **Supercharge your Azure AI Agents with Semantic Kernel Plugins!**

This tutorial shows you how to give your Azure AI agents **superpowers** using Semantic Kernel's plugin architecture!

## What You'll Learn:

1. **Plugin Basics** - What are plugins and why use them?
2. **Simple Plugins** - Creating your first agent plugin
3. **Real-World Examples** - Weather, calculations, and data processing
4. **Advanced Plugins** - Multiple functions and complex logic
5. **Best Practices** - Error handling and plugin design

**Prerequisites:** You should have completed the basic Azure AI Agents tutorial first!

---

## 🛠️ What are Semantic Kernel Plugins?

Imagine your AI agent as a **smart assistant**, but one that can actually **DO things** beyond just chatting using the power of Semantic Kernel!

### Without Plugins:
- Agent can only provide text responses
- Limited to knowledge from training data
- Can't access real-time information
- Can't perform calculations or operations

### With Semantic Kernel Plugins:
- Agent can call external APIs 🌐
- Perform calculations and data processing 🧮
- Access databases and files 📁
- Interact with other services 🔗
- Take actions in the real world! 🚀

### Key Concepts:

- **Plugin Class**: You write Python classes with decorated methods
- **@kernel_function**: Decorator that marks methods as callable functions
- **Function Registration**: Semantic Kernel automatically registers your plugins
- **Function Calling**: The agent decides when and how to use them
- **Function Results**: The agent uses the results to provide better responses

Let's see this in action! ⚡

## 📋 Setup and Prerequisites

Before we start, make sure you have:

1. ✅ Completed the basic Azure AI Agents tutorial
2. ✅ Environment variables set (`AZURE_AI_PROJECT_ENDPOINT`, `AZURE_AI_PROJECT_NAME`)
3. ✅ Azure authentication configured

We'll install any additional packages we need and import everything required for plugin-enabled agents.

In [1]:
# Install required packages
# !pip install semantic-kernel azure-identity requests

# Import everything we need
import os
import asyncio
import json
import requests
import random
from datetime import datetime
from typing import Any, Dict, Annotated

from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.functions import kernel_function

print("📦 All packages imported successfully!")
print("🔧 Ready to create plugin-enabled agents!")

# Verify environment variables
required_vars = ['PROJECT_ENDPOINT', 'MODEL_DEPLOYMENT_NAME']
for var in required_vars:
    if var in os.environ:
        print(f"✅ {var} is set")
    else:
        print(f"❌ {var} is missing!")

📦 All packages imported successfully!
🔧 Ready to create plugin-enabled agents!
✅ AZURE_AI_PROJECT_ENDPOINT is set
✅ AZURE_AI_PROJECT_NAME is set


## 🎯 Your First Plugin: Simple Calculator

Let's start with something simple - a calculator plugin that your agent can use to perform math operations.

**What we'll do:**
1. Write a Python class with calculator methods
2. Use the `@kernel_function` decorator to mark methods as callable
3. Add the plugin to our agent
4. Test it out!

**The magic:** The agent will automatically decide when to use these functions based on user requests!

In [2]:
# Step 1: Create our Calculator Plugin Class
class CalculatorPlugin:
    """A simple calculator plugin for performing basic math operations."""
    
    @kernel_function(
        description="Adds two numbers together",
        name="add_numbers"
    )
    def add_numbers(
        self, 
        a: Annotated[float, "The first number"], 
        b: Annotated[float, "The second number"]
    ) -> Annotated[float, "The sum of the two numbers"]:
        """
        Adds two numbers together.
        
        Args:
            a (float): First number
            b (float): Second number
            
        Returns:
            float: The sum of a and b
        """
        result = a + b
        print(f"🧮 Calculator called: {a} + {b} = {result}")
        return result
    
    @kernel_function(
        description="Subtracts the second number from the first number",
        name="subtract_numbers"
    )
    def subtract_numbers(
        self, 
        a: Annotated[float, "The first number"], 
        b: Annotated[float, "The second number"]
    ) -> Annotated[float, "The difference of the two numbers"]:
        """
        Subtracts the second number from the first number.
        """
        result = a - b
        print(f"🧮 Calculator called: {a} - {b} = {result}")
        return result

# Create an instance of our calculator plugin
calculator_plugin = CalculatorPlugin()

print("✅ Calculator plugin ready!")
print(f"📝 Plugin has addition and subtraction functions available")

✅ Calculator plugin ready!
📝 Plugin has addition and subtraction functions available


In [9]:
# Step 2: Create an agent with our calculator plugin

model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
endpoint = os.environ.get("PROJECT_ENDPOINT")


async def create_calculator_agent():
    """Create an Azure AI agent with a calculator plugin."""

    # Ensure we have the required environment variables
    model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
    endpoint = os.environ.get("PROJECT_ENDPOINT")

    # Create the Azure AI agent client
    client = AzureAIAgent.create_client(credential=DefaultAzureCredential(), 
                                            endpoint=endpoint) 
    
    # Create an agent definition on the Azure AI agent service
    agent_definition = await client.agents.create_agent(
        model=model_deployment_name,
        name="math-genius",
        instructions="You are a helpful math assistant. When users ask for calculations, use the available functions to compute accurate results. Always show your work and explain the calculation."
    )
    
    # Create a Semantic Kernel agent with the calculator plugin
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[calculator_plugin]  # Add our plugin here!
    )
    
    print(f"🤖 Created calculator agent: {agent_definition.name}")
    print(f"🛠️ Agent has calculator plugin with addition and subtraction")
    print(f"🆔 Agent ID: {agent_definition.id}")
    
    return agent, client

# We'll run this in the next cell to test the agent

## 🚀 Testing Our Calculator Agent

Now let's test our plugin-enabled agent! We'll ask it to do some math and watch it automatically use our plugin functions.

**What to expect:**
1. We send a math question
2. The agent recognizes it needs to calculate something
3. The agent calls our calculator plugin functions
4. The agent incorporates the result into its response

**Note:** Semantic Kernel handles all the function calling automatically!

In [10]:
# Test our calculator agent
async def test_calculator_agent():
    agent, client = await create_calculator_agent()
    
    # Test math questions
    test_questions = [
        "Hi! Can you help me calculate 125 + 387? Please show me the exact result.",
        "What is 1000 - 234?",
        "Can you add 45.5 and 12.3 for me?"
    ]
    
    thread = None
    
    try:
        for i, question in enumerate(test_questions, 1):
            print(f"\n{'='*60}")
            print(f"Test {i}: {question}")
            print(f"{'='*60}")
            
            # Use invoke method which handles function calling automatically
            async for response in agent.invoke(messages=question, thread=thread):
                print(f"🤖 {response.name}: {response}")
                thread = response.thread
    
    finally:
        # Cleanup
        if thread:
            await thread.delete()
        await client.agents.delete_agent(agent.id)
        print(f"\n🧹 Calculator agent deleted.")

# Run the test
await test_calculator_agent()

🤖 Created calculator agent: math-genius
🛠️ Agent has calculator plugin with addition and subtraction
🆔 Agent ID: asst_zfCjSEYKmZc6GwDWwQUAoTcH

Test 1: Hi! Can you help me calculate 125 + 387? Please show me the exact result.
🧮 Calculator called: 125.0 + 387.0 = 512.0
🤖 math-genius: Sure! To calculate 125 + 387:

125
+387
-----
512

The exact result is 512.

Test 2: What is 1000 - 234?
🧮 Calculator called: 1000.0 - 234.0 = 766.0
🤖 math-genius: To find 1000 - 234:

1000
- 234
-----
 766

The result is 766.

Test 3: Can you add 45.5 and 12.3 for me?
🧮 Calculator called: 45.5 + 12.3 = 57.8
🤖 math-genius: Sure! Adding 45.5 and 12.3 gives:

45.5 + 12.3 = 57.8

The exact result is 57.8.

🧹 Calculator agent deleted.


---

## 🌟 Level Up: Real-World Plugins

Now let's create more practical plugins that agents can use in real scenarios:

1. **Weather Plugin** - Get current weather information
2. **Random Generator Plugin** - Generate random numbers or choices
3. **Date/Time Plugin** - Get current date and time information
4. **Text Processing Plugin** - Analyze and manipulate text

These examples show how plugins can make your agents incredibly powerful!

In [11]:
# Weather Plugin
class WeatherPlugin:
    """A plugin for getting weather information."""
    
    @kernel_function(
        description="Gets the current weather for a city",
        name="get_weather"
    )
    def get_weather(
        self, 
        city: Annotated[str, "The name of the city to get weather for"]
    ) -> Annotated[str, "The current weather information"]:
        """
        Gets the current weather for a city (mock implementation).
        """
        # In a real implementation, you'd call a weather API
        weather_options = [
            f"Sunny and 72°F in {city}",
            f"Partly cloudy and 68°F in {city}", 
            f"Rainy and 61°F in {city}",
            f"Overcast and 65°F in {city}"
        ]
        weather = random.choice(weather_options)
        print(f"🌤️ Weather check: {weather}")
        return weather

# Random Number Plugin
class RandomPlugin:
    """A plugin for generating random numbers and choices."""
    
    @kernel_function(
        description="Generates a random number between min and max values",
        name="generate_random_number"
    )
    def generate_random_number(
        self, 
        min_val: Annotated[int, "The minimum value (inclusive)"], 
        max_val: Annotated[int, "The maximum value (inclusive)"]
    ) -> Annotated[int, "A random number between min and max"]:
        """
        Generates a random number between min and max values.
        """
        result = random.randint(min_val, max_val)
        print(f"🎲 Generated random number: {result} (between {min_val} and {max_val})")
        return result

# DateTime Plugin
class DateTimePlugin:
    """A plugin for date and time operations."""
    
    @kernel_function(
        description="Gets the current date and time",
        name="get_current_datetime"
    )
    def get_current_datetime(self) -> Annotated[str, "The current date and time"]:
        """
        Gets the current date and time.
        """
        now = datetime.now()
        formatted_time = now.strftime("%Y-%m-%d %H:%M:%S")
        print(f"🕐 Current time: {formatted_time}")
        return formatted_time

# Text Analysis Plugin
class TextAnalysisPlugin:
    """A plugin for analyzing text content."""
    
    @kernel_function(
        description="Analyzes text and returns statistics",
        name="analyze_text"
    )
    def analyze_text(
        self, 
        text: Annotated[str, "The text to analyze"]
    ) -> Annotated[str, "JSON string containing text analysis results"]:
        """
        Analyzes text and returns statistics.
        """
        words = text.split()
        analysis = {
            "character_count": len(text),
            "word_count": len(words),
            "sentence_count": text.count('.') + text.count('!') + text.count('?'),
            "average_word_length": round(sum(len(word) for word in words) / len(words), 2) if words else 0
        }
        print(f"📊 Text analysis complete: {analysis}")
        return json.dumps(analysis)

# Create plugin instances
weather_plugin = WeatherPlugin()
random_plugin = RandomPlugin()
datetime_plugin = DateTimePlugin()
text_analysis_plugin = TextAnalysisPlugin()

print("✅ All utility plugins defined!")
print("🔧 Available plugins: Weather, Random, DateTime, TextAnalysis")

✅ All utility plugins defined!
🔧 Available plugins: Weather, Random, DateTime, TextAnalysis


In [12]:
# Create a super-powered agent with multiple plugins
async def create_super_agent():
    """Create an Azure AI agent with multiple utility plugins."""

    # Ensure we have the required environment variables
    model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
    endpoint = os.environ.get("PROJECT_ENDPOINT")

    # Create the Azure AI agent client
    client = AzureAIAgent.create_client(credential=DefaultAzureCredential(), endpoint=endpoint)

    # Create an agent definition
    agent_definition = await client.agents.create_agent(
        model=model_deployment_name,
        name="super-assistant",
        instructions="""You are a versatile AI assistant with access to multiple plugins. You can:
            - Get weather information for any city
            - Generate random numbers
            - Provide current date and time
            - Analyze text statistics

        Always use the appropriate plugins when users request these capabilities.
        Be helpful, accurate, and explain what you're doing when you use functions."""
    )
    
    
    # Create agent with all plugins
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[
            weather_plugin,
            random_plugin, 
            datetime_plugin,
            text_analysis_plugin
        ]
    )
    
    print(f"🚀 Created super agent: {agent_definition.name}")
    print(f"🛠️ Agent has 4 plugins with multiple functions")
    print(f"🆔 Agent ID: {agent_definition.id}")
    
    return agent, client

print("✅ Super agent creator ready!")

✅ Super agent creator ready!


In [13]:
# Test the super agent with multiple plugin calls
async def test_super_agent():
    agent, client = await create_super_agent()
    
    # Send a complex request that might use multiple plugins
    complex_request = """Hi! I need help with a few things:
    1. What's the weather like in Seattle?
    2. What's the current date and time?
    3. Can you generate a random number between 1 and 100?
    4. Please analyze this text: 'Azure AI Agents with Semantic Kernel are incredibly powerful tools for building intelligent applications!'
    
    Thanks!"""
    
    print(f"💬 Sending complex request with multiple tasks")
    print(f"{'='*80}")
    print(f"👤 USER: {complex_request}")
    print(f"{'='*80}")
    
    thread = None
    function_calls_made = 0
    
    try:
        # Use invoke method which handles all function calling automatically
        async for response in agent.invoke(messages=complex_request, thread=thread):
            print(f"\n🤖 {response.name}:")
            
            # Split long responses for better readability
            content = str(response)
            if len(content) > 500:
                lines = content.split('\n')
                for line in lines:
                    if line.strip():
                        print(f"   {line}")
            else:
                print(f"   {content}")
            
            thread = response.thread
    
    finally:
        # Cleanup
        if thread:
            await thread.delete()
        await client.agents.delete_agent(agent.id)
        print(f"\n🧹 Super agent deleted.")
    
    print(f"\n🌟 Incredible! The agent intelligently used multiple plugins to complete all tasks!")

# Run the super agent test
await test_super_agent()

🚀 Created super agent: super-assistant
🛠️ Agent has 4 plugins with multiple functions
🆔 Agent ID: asst_mEgrXXJ7XqK6p0vJlZsQVCG6
💬 Sending complex request with multiple tasks
👤 USER: Hi! I need help with a few things:
    1. What's the weather like in Seattle?
    2. What's the current date and time?
    3. Can you generate a random number between 1 and 100?
    4. Please analyze this text: 'Azure AI Agents with Semantic Kernel are incredibly powerful tools for building intelligent applications!'

    Thanks!
🌤️ Weather check: Sunny and 72°F in Seattle
🕐 Current time: 2025-06-02 15:37:24
🎲 Generated random number: 29 (between 1 and 100)
📊 Text analysis complete: {'character_count': 105, 'word_count': 14, 'sentence_count': 1, 'average_word_length': 6.57}

🤖 super-assistant:
   Here are the results for your requests:

1. The weather in Seattle is sunny and 72°F.
2. The current date and time is 2025-06-02 15:37:24.
3. Your random number between 1 and 100 is: 29.
4. Text Analysis for: "Azur

---

## 🎯 Practice Exercise: Build Your Own Plugin

Now it's your turn! Let's create a custom plugin and add it to an agent.

**Your Mission:**
1. Create a plugin class with useful methods
2. Use the `@kernel_function` decorator properly
3. Test it with a custom agent

**Plugin Ideas:**
- **Password Generator**: Generate secure passwords
- **Unit Converter**: Convert between different units (temperature, distance, etc.)
- **Word Game**: Create word puzzles or rhymes
- **Advanced Calculator**: More math operations (multiply, divide, square root)
- **Color Palette**: Generate color combinations

**Exercise template below - customize it!**

In [ ]:
# 🎯 YOUR TURN! Create your own plugin

# Example: Password Generator Plugin
class PasswordPlugin:
    """A plugin for generating secure passwords."""
    
    @kernel_function(
        description="Generates a secure password",
        name="generate_password"
    )
    def generate_password(
        self, 
        length: Annotated[int, "Length of the password (default 12)"] = 12,
        include_symbols: Annotated[bool, "Whether to include symbols (default True)"] = True
    ) -> Annotated[str, "The generated password"]:
        """
        Generates a secure password.
        
        Args:
            length (int): Length of the password (default 12)
            include_symbols (bool): Whether to include symbols (default True)
            
        Returns:
            str: Generated password
        """
        import string
        
        characters = string.ascii_letters + string.digits
        if include_symbols:
            characters += "!@#$%^&*"
        
        password = ''.join(random.choice(characters) for _ in range(length))
        print(f"🔐 Generated password of length {length}")
        return password

# TODO: Replace this with your own plugin!
# class YourCustomPlugin:
#     """Your custom plugin description."""
#     
#     @kernel_function(
#         description="What your function does",
#         name="your_function_name"
#     )
#     def your_function(
#         self, 
#         param1: Annotated[str, "Description of param1"]
#     ) -> Annotated[str, "Description of return value"]:
#         """Your function implementation."""
#         # Your logic here
#         return result

# Create plugin instance
password_plugin = PasswordPlugin()

print("✅ Custom plugin ready for testing!")

In [ ]:
# Test your custom plugin
async def test_custom_plugin():
    """Test the custom plugin by creating an agent and invoking it."""

    # Ensure we have the required environment variables
    model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
    endpoint = os.environ.get("PROJECT_ENDPOINT")

    # Create the Azure AI agent client
    client = AzureAIAgent.create_client(credential=DefaultAzureCredential(), endpoint=endpoint)

    # Create agent with your custom plugin
    agent_definition = await client.agents.create_agent(
        model=model_deployment_name,
        name="custom-assistant",
        instructions="You are a helpful assistant with custom capabilities. Use your plugins when users request them. Be friendly and explain what you're doing."
    )
    
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[password_plugin]  # Add your plugin here!
    )
    
    print(f"🎨 Created custom agent: {agent_definition.name}")
    
    # TODO: Customize this message to test your plugin!
    test_message = "Hi! Can you generate a secure password for me? Make it 16 characters long and include symbols."
    
    print(f"\n💬 Test message: {test_message}")
    print(f"{'─'*50}")
    
    thread = None
    
    try:
        # Test the custom plugin
        async for response in agent.invoke(messages=test_message, thread=thread):
            print(f"\n🤖 {response.name}: {response}")
            thread = response.thread
        
        print(f"\n🎉 Great job! Your custom plugin worked perfectly!")
    
    finally:
        # Cleanup
        if thread:
            await thread.delete()
        await client.agents.delete_agent(agent.id)
        print(f"\n🧹 Custom agent deleted.")

# Run the custom plugin test
await test_custom_plugin()

---

## 🏆 Best Practices for Semantic Kernel Plugins

### ✅ Plugin Design:

1. **Clear Names**: Use descriptive function names (`get_weather` not `gw`)
2. **Good Descriptions**: Write clear descriptions in the `@kernel_function` decorator
3. **Proper Annotations**: Use `Annotated` types for better function understanding
4. **Error Handling**: Always handle potential errors gracefully
5. **Return Values**: Return appropriate data types and formats

### ✅ Security Considerations:

1. **Input Validation**: Always validate function parameters
2. **API Keys**: Never hardcode sensitive information
3. **Rate Limiting**: Be mindful of API rate limits
4. **Data Privacy**: Don't log sensitive user data

### ✅ Performance Tips:

1. **Fast Functions**: Keep functions quick to maintain good user experience
2. **Caching**: Cache results when appropriate
3. **Async Operations**: Use async methods when possible for I/O operations
4. **Error Messages**: Provide helpful error messages

### ✅ Testing:

1. **Unit Tests**: Test your plugin methods independently
2. **Edge Cases**: Test with various input combinations
3. **Error Scenarios**: Test how plugins handle errors
4. **Integration**: Test the full agent + plugin workflow

## 🔧 Troubleshooting Common Issues

### Plugin Not Called
**Problem**: Agent doesn't use your plugin
**Solutions**: 
- Check function description clarity in `@kernel_function`
- Verify parameter annotations are descriptive
- Make sure the user request matches plugin capability
- Test with more explicit requests

### Import Errors
**Problem**: Plugin imports fail
**Solutions**:
- Ensure all required packages are installed
- Check that `semantic_kernel` is properly installed
- Verify Azure credentials are configured

### Function Execution Errors
**Problem**: Plugin methods throw exceptions
**Solutions**:
- Add proper error handling in plugin methods
- Validate all input parameters
- Return error messages instead of throwing exceptions
- Log errors for debugging

### Agent Connection Issues
**Problem**: Can't connect to Azure AI services
**Solutions**:
- Verify environment variables are set correctly
- Check Azure authentication (DefaultAzureCredential)
- Ensure proper Azure permissions
- Verify endpoint and model deployment names

### Performance Issues
**Problem**: Plugins are slow
**Solutions**:
- Optimize plugin logic
- Add timeouts to external API calls
- Cache frequently requested data
- Consider async implementations for I/O operations

---

## 🎓 Congratulations! You're Now a Semantic Kernel Plugin Master!

### What You've Accomplished:

✅ **Plugin Fundamentals**:
- Understanding how Semantic Kernel plugins work
- Creating plugin classes with `@kernel_function` decorators
- Using proper type annotations for better AI understanding

✅ **Real-World Examples**:
- Built calculator, weather, random number, and text analysis plugins
- Created multi-plugin agents
- Handled complex conversations with multiple plugin calls

✅ **Best Practices**:
- Learned proper plugin design principles
- Understood security and performance considerations
- Mastered error handling and troubleshooting

### 🚀 Next Level Capabilities:

Now you can create agents that:
- 🌐 **Access External APIs** (weather, news, databases)
- 🧮 **Perform Complex Calculations** (scientific, financial)
- 📊 **Process Data** (analyze files, generate reports)
- 🎮 **Create Interactive Experiences** (games, quizzes)
- 🤖 **Automate Tasks** (send emails, manage calendars)

### 🔮 Advanced Topics to Explore:

1. **Async Plugins**: For better performance with I/O operations
2. **Plugin Chaining**: Have plugins call other plugins
3. **State Management**: Maintain data across plugin calls
4. **External Integrations**: Connect to databases, APIs, and services
5. **Plugin Libraries**: Build reusable plugin collections
6. **Production Deployment**: Scale plugin-enabled agents

### 💡 Key Takeaways:

- **Plugins transform agents from chatbots into action-takers**
- **Semantic Kernel makes plugin development elegant and powerful**
- **Good plugin design makes agents more intelligent and useful**
- **Error handling and testing are crucial for reliable agents**
- **The possibilities are endless with creative plugin combinations**

**Keep building amazing plugin-powered agents with Semantic Kernel!** 🌟

---

*Remember: With great plugin power comes great responsibility! Always consider security, privacy, and user experience when designing agent plugins.*

## 📚 Quick Reference

### Plugin Class Template:
```python
class MyPlugin:
    """Description of what this plugin does."""
    
    @kernel_function(
        description="Clear description of what the function does",
        name="my_function"
    )
    def my_function(
        self, 
        param1: Annotated[str, "Description of param1"],
        param2: Annotated[int, "Description of param2"] = 10
    ) -> Annotated[str, "Description of return value"]:
        """Function implementation."""
        # Your logic here
        return result
```

### Agent Creation Template:
```python
async def create_agent_with_plugins():
    async with DefaultAzureCredential() as creds:
        async with AzureAIAgent.create_client(credential=creds) as client:
            agent_definition = await client.agents.create_agent(
                model=AzureAIAgentSettings().model_deployment_name,
                name="my-agent",
                instructions="Agent instructions here"
            )
            
            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
                plugins=[plugin1, plugin2, plugin3]
            )
            
            return agent, client
```

### Plugin Testing Template:
```python
async def test_agent():
    agent, client = await create_agent_with_plugins()
    
    thread = None
    
    try:
        async for response in agent.invoke(messages="Your test message", thread=thread):
            print(f"🤖 {response.name}: {response}")
            thread = response.thread
    
    finally:
        if thread:
            await thread.delete()
        await client.agents.delete_agent(agent.id)
```

## 🔗 Additional Resources

### Documentation:
- [Semantic Kernel Documentation](https://learn.microsoft.com/semantic-kernel/)
- [Azure AI Agents Documentation](https://docs.microsoft.com/azure/ai-services/)
- [Plugin Development Guide](https://learn.microsoft.com/semantic-kernel/)

### Examples to Try:
- **File Processing**: Plugins that read/write files
- **Database Operations**: Connect to databases
- **Image Processing**: Analyze or manipulate images
- **Email/SMS**: Send notifications
- **Web Scraping**: Get data from websites
- **Machine Learning**: Run ML models as plugins

### Community:
- Share your plugins with the community
- Learn from other developers' implementations
- Contribute to open-source plugin libraries

**Happy coding with Azure AI Agents and Semantic Kernel Plugins!** 🎉